In [12]:
import h5py
import numpy as np


with h5py.File("../data/piecewise_linear_3/bayesopt.hdf5") as hdf5_file:
    n_completed_bayesopt_steps = hdf5_file["/"].attrs["n_completed_bayesopt_steps"]
    groups = [
        *[f"bayesopt/{i+1}" for i in range(n_completed_bayesopt_steps - 1)],
        "initialisation"
    ]
    
    previous_ecrh_parameters = np.array([hdf5_file[g]["ecrh_parameters"] for g in groups])

    previous_ecrh_parameters = previous_ecrh_parameters.reshape((-1, previous_ecrh_parameters.shape[-1]))
    
    new_ecrh_parameters = hdf5_file[f"bayesopt/{n_completed_bayesopt_steps}"]["ecrh_parameters"][:]

In [13]:
from scipy.spatial import KDTree

def get_nearest_neigbour(new_parameters, p):
    x = np.linspace(0, 1, 100)
    tree = KDTree([piecewise_linear_3(x, previous_parameters) for previous_parameters in previous_ecrh_parameters])
    distances, indices = tree.query([piecewise_linear_3(x, parameters) for parameters in new_parameters], p=p)
    return previous_ecrh_parameters[indices]

array([[0.4578093 , 0.35124235, 0.10005116, 0.46268437, 0.41887409,
        0.15673636, 0.86206622, 0.49014452, 0.44402621, 0.87103851,
        0.29881602, 0.15742992],
       [0.99257309, 0.9998908 , 0.78344723, 0.50309421, 0.75971751,
        0.59304957, 0.        , 0.37924851, 0.68294241, 0.02268925,
        0.67592167, 0.44669429],
       [0.99134753, 0.98022958, 0.67278627, 0.19213999, 0.        ,
        0.52043002, 0.38602904, 0.71378378, 0.25018474, 0.8756347 ,
        0.        , 0.72960503],
       [0.02834003, 0.72016944, 0.54925319, 0.22654173, 0.77863737,
        0.05721656, 0.51698759, 0.58353709, 0.04894107, 0.02485641,
        0.98485638, 0.51394297],
       [0.02834003, 0.72016944, 0.54925319, 0.22654173, 0.77863737,
        0.05721656, 0.51698759, 0.58353709, 0.04894107, 0.02485641,
        0.98485638, 0.51394297],
       [0.21053921, 0.79692285, 0.02885059, 0.46211604, 0.41592912,
        0.61813042, 0.05689216, 0.5685317 , 0.52334437, 0.06432417,
        0.06028311,

In [18]:
import plotly.graph_objects as go 
from jetto_mobo.ecrh import piecewise_linear_3
import plotly

for p in [1, 2, np.inf]:
    figure = go.Figure()
    x = np.linspace(0, 1, 100)
    nearest_neighbours = get_nearest_neigbour(new_ecrh_parameters, p)
    for i, (new, old) in enumerate(zip(new_ecrh_parameters, nearest_neighbours)):
        figure.add_traces(
            [
                go.Scatter(
                    x=x,
                    y=piecewise_linear_3(x, old),
                    mode='lines',
                    name="Closest previous ECRH function",
                    line_color=plotly.colors.qualitative.Alphabet[i],
                    legendgroup=str(i),
                ),
                go.Scatter(
                    x=x,
                    y=piecewise_linear_3(x, new),
                    mode='lines',
                    line_dash='dash',
                    name="New ECRH function",
                    line_color=plotly.colors.qualitative.Alphabet[i],
                    legendgroup=str(i),
                    showlegend=False,
                ),
            ]
        )
    figure.update_layout(
        template="simple_white",
        xaxis_title="Normalised radius",
        yaxis_title="Normalised ECRH power density",
        title=f"Nearest neighbour (l-{p} norm)",)
    figure.show()